## CRAWL DARI DETIK

### Def untuk crawling dari Portal DETIK
Note : 
- ganti User-Agent pada headers -> ke user agent browser sendiri

In [ ]:
import os
import time
import csv
import requests
from bs4 import BeautifulSoup

def detik(keyword, n_berita):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
    }

    save_dir = r"/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/corpus/detik"

    data_detik = []
    page = 1

    while len(data_detik) < n_berita:
        alamatURL = f'https://www.detik.com/search/searchall?query={keyword}&sortby=time&page={page}'
        req = requests.get(alamatURL, headers=headers)
        print(f"\n🔄 Scraping halaman {page}...")

        if req.status_code != 200:
            print("❌ Gagal mengakses halaman.")
            break

        soup = BeautifulSoup(req.content, 'lxml')
        articles = soup.find_all('article')

        if not articles:
            print("❌ Tidak menemukan artikel lagi.")
            break

        for article in articles:
            try:
                a_tag = article.find('a', href=True)
                if not a_tag:
                    continue

                url = a_tag['href']
                title_tag = article.find('h3', class_='media__title')
                judul = title_tag.get_text(strip=True) if title_tag else a_tag.get_text(strip=True)

                date_tag = article.find('span', class_='date')
                tanggal = date_tag.get_text(strip=True).split(',')[-1].strip() if date_tag else "?"

                detail_req = requests.get(url, headers=headers)
                soup_detail = BeautifulSoup(detail_req.content, 'lxml')
                body = soup_detail.find('div', class_='detail__body-text itp_bodycontent')
                berita = body.get_text(separator=' ', strip=True) if body else "Tidak ditemukan"

                if berita and len(berita) > 30:
                    data_detik.append({
                        'judul': judul,
                        'berita': berita,
                        'url': url,
                        'tanggal': tanggal
                    })
                    print(f"[{len(data_detik)}] {judul}")

                    if len(data_detik) >= n_berita:
                        break

                time.sleep(1)

            except Exception as e:
                print("⚠️ Error ambil artikel:", e)
                continue

        page += 1

    if data_detik:
        filename = os.path.join(save_dir, f"detik_news_{keyword.lower().replace(' ','')}_{time.strftime('%Y%m%d_%H%M%S')}.csv")
        with open(filename, "w", newline="", encoding="utf-8-sig") as f:
                writer = csv.DictWriter(f, fieldnames=["judul", "berita", "url"])
                writer.writeheader()
                writer.writerows(data_detik)

        print(f"/n✅ {len(data_detik)} artikel '{keyword}' disimpan ke {filename}")
    else:
            print(f"❌ Tidak ada data valid untuk '{keyword}'")

In [ ]:
detik(keyword="IHSG", n_berita=100)